# Analyse

Warnungen des Systems ignorieren.

In [1]:
import warnings
warnings.filterwarnings('ignore')

Einalden der benötigen Bibliotheken.

In [2]:
from shapely.geometry import *
from shapely.ops import unary_union

import geopandas as gpd
import pandas as pd
import numpy as np

Für das Anzeigen des GDF wird noch eingestellt über Pandas, dass nicht die wissenschaftliche Schreibweise angezeigt wird, sondern Gleitkommazahlen mit 4 Nachkommastellen

In [3]:
pd.options.display.float_format = '{:.4f}'.format

In [4]:
kreise_gdf = gpd.read_file("./Daten/Bevoelkerung/Anteil_Ue64_SH.geojson")
ewz_gdf = gpd.read_file("./Daten/Bevoelkerung/Gemeindegrenzen_SH_2021_mit_Einwohnerzahl.geojson")

Gesamtfläche von Schleswig-Holstein in km2 berechnen

In [5]:
gesamtflaeche = kreise_gdf['geometry'].area.sum()
gesamtflaeche/1000000

15613.633027296251

Werte für die Spalte ai0207 (Anteil Ü64) mit 2222222222.0000 haben keine Angaben. Damit diese Zahl nicht in der statistischen Auswertung berücsichtigt wird, werden die Werte mit NaN ersetzt. 

In [8]:
kreise_gdf['ai0207'] = kreise_gdf['ai0207'].replace([2222222222.0000], np.NaN)

Da der Schlüssel von kreide_gdf und AGS von ewz_gdf nicht 100% übereinstimmen, müssen Anpassungen vorgenommen werden. Die Schlüsselzahl bei kreise_gdf besteht nur aus 5 Zahlen, der bei ewz_gdf aus 8. Dies ist nur bei den Städten der Fall, von daher können hier die fehlenden drei Nullen angehängt werden.

In [9]:
# Funktion zum Anhängen von "000"
def append_zeros(value):
    if len(str(value)) < 8:
        return str(value) + "000"
    else:
        return str(value)

# Anwenden der Funktion auf die gewünschte Spalte
kreise_gdf['schluessel'] = kreise_gdf['schluessel'].apply(append_zeros)

Auf Grundlage der Schlüssel für die Gemeinden (schluessel und AGS) könne die Einwohnerzahlen nun über eine Vereinigung zu dem kreise_gdf hinzugefügt werden.

In [10]:
merged_gdf = kreise_gdf.merge(ewz_gdf[['GEN','EWZ','AGS']], left_on='schluessel', right_on = 'AGS', how="left")

### Isochronen einladen 

In [11]:
iso = gpd.read_file("./Isochronen/Minimalgraph/IST/Alpha0002/20minuten.geojson")

### Alternativ mit Rastern

In [ ]:
raster = gpd.read_file("./Isochronen/Minimalgraph/IST/Raster500/rastermittraveltim550.geojson")
# raster nur für 5Minuten behalten
# Filtern nach Spalte 'mean_travel_time'
raster = raster[raster['mean_travel_time'] <= 20]

### Analyse

Funktion, die die Isochronen zu einer Geomterie machen und dann die Überlappungen mit den einzelnen Gemeinden berechnet (als Fläche absolut, relativ und Anzahl der Einwohner, die abgedeckt werden,ebenfalls relativ und absolut). 

In [12]:
def pro_gemeinde(datensatz):
    # Einzelnen Isochronen zu einer einzelnen Geometrie zu einem Multipolygon vereinigen 
    # Verwende die unary_union-Funktion, um alle Geometrien zu einem einzigen Multipolygon zu vereinen
    combined_geometry = unary_union(datensatz['geometry'])

    # Erstelle eine neue GeoDataFrame mit dem kombinierten Multipolygon
    gdf = gpd.GeoDataFrame({'combined_geometry': [combined_geometry]}, geometry='combined_geometry')

    # Überlappungen der Isochronen mit den einzelnen Gemeidnen berechnen. Dafür wird eine neue Spalte 'coverage_percentage' 
    # zu GeoDataFrame merged_gdf hinzugefügt. In diesem steht die Überalppung in Prozent.
    # Berechne die Überlappung zwischen Isochronen und Gemeinden
    intersection = gpd.overlay(merged_gdf, gdf, how='intersection')

    # Berechne die Fläche der überlappenden Geometrien
    intersection['overlap_area'] = intersection.geometry.area

    # Fasse die Fläche nach Gemeinde zusammen
    overlap_by_gemeinde = intersection.groupby('schluessel')['overlap_area'].sum()

    # Füge die überlappende Fläche den Gemeinden hinzu
    new_gdf = merged_gdf.merge(overlap_by_gemeinde, how='left', left_on='schluessel', right_index=True)

    # Berechne den Prozentsatz der Gemeindefläche, die abgedeckt wird
    new_gdf['coverage_percentage'] = (new_gdf['overlap_area'] / new_gdf['geometry'].area) * 100

    # Einwohnerzahl der Gemeinden ebenfalls noch in Prozent zu gesamt SH umrechnen. Dafür muss die Einwohnerzahl EWZ 
    # durch die Gesamteinwohnerzahl von SH geteilt werden und mit 100 multipliziert.
    new_gdf['EWZ'].sum()
    # Berechne den Prozentsatz der Gemeindefläche, die abgedeckt wird
    new_gdf['EWZ_percentage'] = (new_gdf['EWZ'] / new_gdf['EWZ'].sum()) * 100

    # Einwohner in Prozent, die eine Arzt in 5 Minuten erreichen
    # Berechne den Prozentsatz der Einwohner pro Gemeinde, die abgedeckt sind durch Isochronen
    new_gdf['anzahl_EW_abgedeckt'] = new_gdf['EWZ'] * (new_gdf['coverage_percentage'] / 100)
    
    return (new_gdf)

In [13]:
iso = pro_gemeinde(iso)

Gesamte Fläche in km2 die innerhalb die die Isochrone abdeckt.

In [14]:
iso['overlap_area'].sum()/1000000

372.98838732506516

Einwohneranzahl, die innerhalb der Isochrone liegen und einen Arzt erreichen.

In [104]:
iso['anzahl_EW_abgedeckt'].sum()

54729.950423162176

Einwohneranzahl gesamt in Prozent (zu ganz SH im Vergleich), die innerhlab der Isochrone liegen und einen Arzt erreichen.

In [105]:
((iso['anzahl_EW_abgedeckt'].sum()) / (iso['EWZ'].sum())) * 100

1.8730272680287054